In [ ]:
!pip install magenta

     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 352 kB 36.3 MB/s 
     |████████████████████████████████| 69 kB 6.4 MB/s 
     |████████████████████████████████| 254 kB 42.9 MB/s 
     |████████████████████████████████| 210 kB 47.6 MB/s 
     |████████████████████████████████| 3.6 MB 35.7 MB/s 
     |████████████████████████████████| 5.6 MB 61.4 MB/s 
     |████████████████████████████████| 90 kB 5.6 MB/s 
     |████████████████████████████████| 204 kB 38.6 MB/s 
     |████████████████████████████████| 1.4 MB 37.6 MB/s 
     |████████████████████████████████| 1.6 MB 31.8 MB/s 
     |████████████████████████████████| 2.3 MB 33.1 MB/s 
     |████████████████████████████████| 20.2 MB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 44.0 MB/s 
     |████████████████████████████████| 5.8 MB 37.5 MB/s 
     |████████████████████████████████| 367 kB 54.3 MB/s 
     |████████████████████████████████| 981 kB 45.3 MB/s 
     |█████████████

In [ ]:
import os
import tensorflow as tf
from six.moves import urllib
def download_checkpoint(model_name: str,checkpoint_name: str,target_dir: str):
              tf.io.gfile.makedirs(target_dir)
              
              checkpoint_target = os.path.join(target_dir, checkpoint_name)
              if not os.path.exists(checkpoint_target):
                response = urllib.request.urlopen(
                  f"https://storage.googleapis.com/magentadata/models/"
                  f"{model_name}/checkpoints/{checkpoint_name}")
                data = response.read()
                local_file = open(checkpoint_target, 'wb')
                local_file.write(data)
                local_file.close()

In [ ]:
from magenta.models.music_vae import TrainedModel, configs

def get_model(name: str):

  checkpoint = name + ".tar"
  download_checkpoint("music_vae", checkpoint, "checkpoints")
  return TrainedModel(

    configs.CONFIG_MAP[name.split(".")[0] if "." in name else name],
    batch_size=8,
    checkpoint_dir_or_path=os.path.join("checkpoints", checkpoint))


/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [ ]:
!pip install visual_midi

In [ ]:
import os
import time
from typing import Union, List, Optional

import magenta.music as mm
from note_seq.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter


def save_midi(sequences: Union[NoteSequence, List[NoteSequence]],
              output_dir: Optional[str] = None,
              prefix: str = "sequence"):

  output_dir = os.path.join("output", output_dir) if output_dir else "output"
  os.makedirs(output_dir, exist_ok=True)
  if not isinstance(sequences, list):
    sequences = [sequences]
  for (index, sequence) in enumerate(sequences):
    date_and_time = time.strftime("%Y-%m-%d_%H%M%S")
    filename = f"{prefix}_{index:02}_{date_and_time}.mid"
    path = os.path.join(output_dir, filename)
    mm.midi_io.note_sequence_to_midi_file(sequence, path)
    print(f"Generated midi file: {os.path.abspath(path)}")


def save_plot(sequences: Union[NoteSequence, List[NoteSequence]],
              output_dir: Optional[str] = None,
              prefix: str = "sequence",
              **kwargs):

  output_dir = os.path.join("output", output_dir) if output_dir else "output"
  os.makedirs(output_dir, exist_ok=True)
  if not isinstance(sequences, list):
    sequences = [sequences]
  for (index, sequence) in enumerate(sequences):
    date_and_time = time.strftime("%Y-%m-%d_%H%M%S")
    filename = f"{prefix}_{index:02}_{date_and_time}.html"
    path = os.path.join(output_dir, filename)
    midi = mm.midi_io.note_sequence_to_pretty_midi(sequence)
    plotter = Plotter(**kwargs)
    plotter.save(midi, path)
    print(f"Generated plot file: {os.path.abspath(path)}")

In [ ]:
from typing import List
from note_seq.protobuf.music_pb2 import NoteSequence

def sample(model_name: str,
           num_steps_per_sample: int) -> List[NoteSequence]:

  model = get_model(model_name)
  sample_sequences = model.sample(n=2, length=num_steps_per_sample)
  save_midi(sample_sequences, "sample", model_name)
  save_plot(sample_sequences, "sample", model_name)

  return sample_sequences

In [ ]:
from magenta.music import DEFAULT_STEPS_PER_BAR
num_bar_per_sample = 2
num_steps_per_sample = num_bar_per_sample * DEFAULT_STEPS_PER_BAR 
generated_sample_sequences = sample("cat-mel_2bar_big", num_steps_per_sample)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.
INFO:tensorflow:Unbundling checkpoint.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Restoring parameters from /tmp/tmpp6f2y3cz/cat-mel_2bar_big.ckpt
Generated midi file: /content/output/sample/cat-mel_2bar_big_00_2022-05-02_165707.mid
Generated midi file: /content/output/sample/cat-mel_2bar_big_01_2022-05-02_165707.mid
Generated plot file: /content/output/sample/cat-mel_2bar_big_00_2022-05-02_165707.html
Generated plot file: /content/output/sample/cat-mel_2bar_big_01_2022-05-02_165708.html


In [ ]:
def interpolate(model_name: str,
                sample_sequences: List[NoteSequence],
                num_steps_per_sample: int,
                num_output: int,
                total_bars: int) -> NoteSequence:

  if len(sample_sequences) != 2:
    raise Exception(f"Wrong number of sequences, "
                    f"expected: 2, actual: {len(sample_sequences)}")
  if not sample_sequences[0].notes or not sample_sequences[1].notes:
    raise Exception(f"Empty note sequences, "
                    f"sequence 1 length: {len(sample_sequences[0].notes)}, "
                    f"sequence 2 length: {len(sample_sequences[1].notes)}")

  model = get_model(model_name)

  interpolate_sequences = model.interpolate(
    start_sequence=sample_sequences[0],
    end_sequence=sample_sequences[1],
    num_steps=num_output,
    length=num_steps_per_sample)

  save_midi(interpolate_sequences, "interpolate", model_name)
  save_plot(interpolate_sequences, "interpolate", model_name)


  interpolate_sequence = mm.sequences_lib.concatenate_sequences(
    interpolate_sequences, [4] * num_output)

  save_midi(interpolate_sequence, "merge", model_name)
  save_plot(interpolate_sequence, "merge", model_name,
            plot_max_length_bar=total_bars,
            bar_fill_alphas=[0.50, 0.50, 0.05, 0.05])

  return interpolate_sequence


In [ ]:
num_output = 10
total_bars = num_output * num_bar_per_sample
generated_interpolate_sequence = interpolate("cat-mel_2bar_big",
                  generated_sample_sequences,
                  num_steps_per_sample,
                  num_output,
                  total_bars)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


INFO:tensorflow:Unbundling checkpoint.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Restoring parameters from /tmp/tmpvl1ars9e/cat-mel_2bar_big.ckpt
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_00_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_01_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_02_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_03_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_04_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_05_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_06_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_07_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_08_2022-05-02_165731.mid
Generated midi file: /content/output/interpolate/cat-mel_2bar_big_09_2022-05-02_165731.mid
Generated

Generated plot file: /content/output/interpolate/cat-mel_2bar_big_02_2022-05-02_165731.html
Generated plot file: /content/output/interpolate/cat-mel_2bar_big_03_2022-05-02_165731.html
Generated plot file: /content/output/interpolate/cat-mel_2bar_big_04_2022-05-02_165731.html


Generated plot file: /content/output/interpolate/cat-mel_2bar_big_05_2022-05-02_165732.html


Generated plot file: /content/output/interpolate/cat-mel_2bar_big_06_2022-05-02_165732.html
Generated plot file: /content/output/interpolate/cat-mel_2bar_big_07_2022-05-02_165732.html
Generated plot file: /content/output/interpolate/cat-mel_2bar_big_08_2022-05-02_165732.html


Generated plot file: /content/output/interpolate/cat-mel_2bar_big_09_2022-05-02_165732.html
Generated midi file: /content/output/merge/cat-mel_2bar_big_00_2022-05-02_165732.mid


Generated plot file: /content/output/merge/cat-mel_2bar_big_00_2022-05-02_165732.html
